In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk import word_tokenize
from sklearn.pipeline import make_pipeline
nltk.download('punkt')

from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate


import random
import torch
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda


In [2]:
resumeDataSet = pd.read_csv('/content/drive/MyDrive/dataset_lm.csv' ,encoding='utf-8')

In [3]:
label_dict = {'primary_teacher' : 6,
              'kg_teacher' : 3,
              'esl_teacher' : 1,
              'ict_teacher' : 2,
              'science_teacher' : 7,
              'music_teacher' : 5,
              'math_teacher' : 4,
              'art_teacher' : 0
              }

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
train, test = train_test_split(resumeDataSet, test_size=0.2,random_state=42, stratify=resumeDataSet['position'])

train_df, valid_df = train_test_split(train, test_size=0.2, random_state=42)
X_train, y_train = train_df["cleaned_resume"], train_df["position"]
X_valid, y_valid = valid_df["cleaned_resume"], valid_df["position"]

##**BOW**

In [5]:
def return_scores(model, X_train, y_train, grams, **kwargs):

    scoring = ["accuracy",
            "precision_weighted",
            "recall_weighted", 
            "f1_weighted"]
    entries = []
    model_name = model.__class__.__name__
    scores = cross_validate(model, 
                            X_train, 
                            y_train, 
                            scoring = scoring, 
                            **kwargs)
    all_scores = []
    for key, value in scores.items():
        all_scores.append(value.mean())
    entries.append((model_name, all_scores[2], all_scores[3], all_scores[4], all_scores[5], grams))
    cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'grams' ])
    return cv_df

In [6]:
def cross_validate_models(embedding_function, X_train, y_train, grams):
    pipe_lr = make_pipeline(embedding_function, LogisticRegression(max_iter=1000, random_state=2))
    pipe_svc = make_pipeline(embedding_function, LinearSVC(random_state=2))
    pipe_knn = make_pipeline(embedding_function, KNeighborsClassifier())
    pipe_rf = make_pipeline(embedding_function, RandomForestClassifier(random_state=2))
    pipe_mnb = make_pipeline(embedding_function, MultinomialNB())
    pipe_xgb = make_pipeline(embedding_function, XGBClassifier())
    classifiers = {
        'Logistic Regression' : pipe_lr,
        "LinearSVC": pipe_svc,
        'KNN' : pipe_knn,
        'Random Forest' : pipe_rf,
        'MultinomialNB' : pipe_mnb,
        'XGBoost' : pipe_xgb
    }

    # cross validation score of the default model hyperparameter
    results_df = pd.DataFrame()
    for (name, pipe) in classifiers.items():
        results_df = results_df.append(return_scores(pipe, X_train, y_train, grams))
        results_df = results_df.replace('Pipeline',name)
    return results_df


In [7]:
# preprocessor to remove digits in string
def remove_numeric_string(text):
    return re.sub(r'\d+', '', text)


In [22]:
results = pd.DataFrame(columns=['model_name', 'accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'grams' ])

In [ ]:
for grams in [(1,2),(1,3),(2,2),(2,3), (3,3)]:
      bow_vectorizer = CountVectorizer(dtype=np.float32,
                                        preprocessor=remove_numeric_string, # remove numeric term
                                        strip_accents='unicode',
                                        tokenizer=word_tokenize,
                                        ngram_range=grams, # keep the unigram
                                        analyzer='word', # feature should be made of word n-gram
                                        min_df=10, # ignore terms appeared less than 10 times
                                        max_df=0.8, # ignore terms appeared more than 80% of the tweets available
                                       ) 
      bow_vector = bow_vectorizer.fit_transform(train['cleaned_resume'])
      #bow_features = pd.DataFrame(bow_vector.toarray(), columns=sorted(bow_vectorizer.vocabulary_))
      #result = pd.read_csv('/content/drive/MyDrive/BOW_models/result_bow.csv' ,encoding='utf-8')
      bow_results = results.append(cross_validate_models(bow_vectorizer, X_train, y_train, grams))
      #result.to_csv('/content/drive/MyDrive/BOW_models/result_bow.csv' ,encoding='utf-8')

In [ ]:
bow_results[['model_name', 'f1_weighted', 'grams']]

,model_name,f1_weighted,grams
0,Logistic Regression,0.842928,"(2, 3)"
1,LinearSVC,0.837160,"(2, 3)"
2,KNN,0.354397,"(2, 3)"
3,Random Forest,0.864920,"(2, 3)"
4,MultinomialNB,0.743271,"(2, 3)"
5,XGBoost,0.871313,"(2, 3)"
6,Logistic Regression,0.872614,"(1, 2)"
7,LinearSVC,0.872447,"(1, 2)"
8,KNN,0.567256,"(1, 2)"
9,Random Forest,0.903242,"(1, 2)"


In [ ]:
bow_results.max()

model_name             XGBoost
accuracy              0.927534
precision_weighted    0.928296
recall_weighted       0.927534
f1_weighted           0.927507
grams                   (3, 3)
dtype: object

##**TF-IDF**

In [ ]:
for grams in [(1,2),(1,3),(2,2),(2,3), (3,3)]:
      tfidf_vectorizer = TfidfVectorizer(dtype=np.float32,
                                        preprocessor=remove_numeric_string, # remove numeric term
                                        strip_accents='unicode',
                                        tokenizer=word_tokenize,
                                        ngram_range=grams, # keep the unigram
                                        analyzer='word', # feature should be made of word n-gram
                                        min_df=10, # ignore terms appeared less than 10 times
                                        max_df=0.8, # ignore terms appeared more than 80% of the tweets available
                                        sublinear_tf=True) 
      tfidf_vector = tfidf_vectorizer.fit_transform(train['cleaned_resume'])
      tfidf_results = results.append(cross_validate_models(tfidf_vectorizer, X_train, y_train, grams))

In [ ]:
tfidf_results

,Unnamed: 0,model_name,accuracy,precision_weighted,recall_weighted,f1_weighted,grams
14,14,KNN,0.630882,0.630002,0.630882,0.626278,"(2, 2)"
2,2,KNN,0.637321,0.642809,0.637321,0.632505,"(1, 2)"
26,0,KNN,0.161223,0.466559,0.161223,0.108273,"(3, 3)"
20,20,KNN,0.608875,0.608480,0.608875,0.603640,"(2, 3)"
8,8,KNN,0.627843,0.631366,0.627843,0.623049,"(1, 3)"
1,1,LinearSVC,0.898328,0.899797,0.898328,0.898202,"(1, 2)"
25,0,LinearSVC,0.696121,0.701141,0.696121,0.697431,"(3, 3)"
7,7,LinearSVC,0.898710,0.900754,0.898710,0.898737,"(1, 3)"
19,19,LinearSVC,0.850527,0.852321,0.850527,0.850553,"(2, 3)"
13,13,LinearSVC,0.858872,0.860448,0.858872,0.858842,"(2, 2)"


##**Word2Vec**

In [ ]:
!pip install gensim==4.3.0
!pip install numpy==1.24.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20443 sha256=dfef508b28acc02971adefecd484b7b527d11b96110a5a30b558be198a9b8177
  Stored in directory: /root/.cache/pip/wheels/6a/65/c4/d27eeee9ba3fc150a0dae150519591103b9e0dbffde3ae77dc
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3530 sha256=5bf5097f7455c184adba03cc018baf58bb7607981c449ffa85046f018c6be403
  Stored in directory: /root/.cache/pip/wheels/ba/d9/a0/ddd93af16d5855dd9bad417623e70948fdac119d1d34fb17c8
Successfully built fst-pso miniful
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0

In [ ]:
import gensim
from gensim.models import Word2Vec
from collections import defaultdict


In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = next(iter(word2vec.items()))[1].shape[0]

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ], dtype=object)

In [ ]:
word2vec_model = Word2Vec(X_train.apply(word_tokenize), seed=1)
word2vec_dict = dict(zip(word2vec_model.wv.key_to_index.keys(), 
         word2vec_model.wv.vectors))

word2vec_results = cross_validate_models(MeanEmbeddingVectorizer(word2vec_dict), 
                                         X_train.apply(word_tokenize), y_train, grams=1)

word2vec_results

,model_name,accuracy,precision_weighted,recall_weighted,f1_weighted,grams
0,Logistic Regression,0.697261,0.703468,0.697261,0.698142,1
0,LinearSVC,0.723059,0.723010,0.723059,0.719981,1
0,KNN,0.503789,0.509363,0.503789,0.498503,1
0,Random Forest,0.592930,0.599654,0.592930,0.594634,1
0,MultinomialNB,NaN,NaN,NaN,NaN,1
0,XGBoost,0.608485,0.616932,0.608485,0.611073,1


In [ ]:
corpus = dtf_train["text_clean"]

## create list of lists of unigrams
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)

## detect bigrams and trigrams
bigrams_detector = gensim.models.phrases.Phrases(lst_corpus, 
                 delimiter=" ".encode(), min_count=5, threshold=10)
bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)
trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[lst_corpus], 
            delimiter=" ".encode(), min_count=5, threshold=10)
trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

In [ ]:
word2vec_results.to_csv('/content/drive/MyDrive/word2vec_models/wv1.csv' ,encoding='utf-8')

In [ ]:
from gensim.models import KeyedVectors
path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
word2vec_pretrained = KeyedVectors.load_word2vec_format(path,binary=True)
word2vec_pretrained_dict = dict(zip(word2vec_pretrained.key_to_index.keys(), 
                                    word2vec_pretrained.vectors))

pretrained_word2vec_results = cross_validate_models(MeanEmbeddingVectorizer(word2vec_pretrained_dict), 
                                         X_train.apply(word_tokenize), y_train, grams=2)
pretrained_word2vec_results.to_csv('/content/drive/MyDrive/word2vec_models/wv2.csv' ,encoding='utf-8')

KeyboardInterrupt: ignored

In [ ]:
pretrained_word2vec_results

,model_name,accuracy,precision_weighted,recall_weighted,f1_weighted,grams
0,Logistic Regression,0.624425,0.640363,0.624425,0.627439,1
0,LinearSVC,0.705998,0.707068,0.705998,0.703216,1
0,KNN,0.492776,0.526306,0.492776,0.477034,1
0,Random Forest,0.582321,0.588119,0.582321,0.581797,1
0,MultinomialNB,NaN,NaN,NaN,NaN,1
0,XGBoost,0.614181,0.623035,0.614181,0.616170,1
